In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adamax

from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix


In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input,zoom_range=0.1,brightness_range=[0.5,1.3],
                                   width_shift_range=0.1,height_shift_range=0.1,validation_split=0.1)
test_datagen=ImageDataGenerator(preprocessing_function=tf.keras.applications.xception.preprocess_input)

In [6]:
df_meta=pd.read_csv("Chest_xray_Corona_Metadata.csv",index_col=0)
df_meta.head()

,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN
1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN
2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN
3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN
4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN


In [7]:
BATCH_SIZE=64
path="Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/"
train_images=train_datagen.flow_from_dataframe(dataframe=df_meta[df_meta["Dataset_type"]=="TRAIN"],x_col='X_ray_image_name',y_col='Label',
                                               color_mode='rgb', class_mode='categorical',target_size=(150,150),
                                               batch_size=BATCH_SIZE,shuffle=True,seed=1234,subset='training', 
                                               directory=path+"/train")

val_images=train_datagen.flow_from_dataframe(dataframe=df_meta[df_meta["Dataset_type"]=="TRAIN"],x_col='X_ray_image_name',y_col='Label',color_mode='rgb',
                                             class_mode='categorical',target_size=(150,150),batch_size=BATCH_SIZE,shuffle=True,seed=1234,
                                             subset='validation', directory=path+"/train")

test_images = test_datagen.flow_from_dataframe(dataframe=df_meta[df_meta["Dataset_type"]=="TEST"],x_col='X_ray_image_name',y_col='Label',
                                               color_mode='rgb',class_mode='categorical',target_size=(150,150),batch_size=BATCH_SIZE,shuffle=False,
                                               directory=path+"/test")



Found 4758 validated image filenames belonging to 2 classes.
Found 528 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [8]:
inputs = tf.keras.layers.Input((150,150,3))
base_model=tf.keras.applications.xception.Xception(include_top=False, weights="imagenet",input_shape=(150,150,3), pooling='avg') 
x=base_model(inputs)
output=layers.Dense(2, activation='sigmoid')(x)
model=tf.keras.models.Model(inputs=inputs, outputs=output)

In [9]:
model.compile(Adamax(learning_rate=1e-4), loss='binary_crossentropy',metrics=['accuracy'])
history = model.fit(train_images, validation_data=val_images, epochs=30)

Epoch 1/30


C:\Users\echel\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 353s 4s/step - accuracy: 0.8558 - loss: 0.4633 - val_accuracy: 0.4754 - val_loss: 1.0222
Epoch 2/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 316s 4s/step - accuracy: 0.9561 - loss: 0.1374 - val_accuracy: 0.8333 - val_loss: 0.3505
Epoch 3/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 318s 4s/step - accuracy: 0.9705 - loss: 0.0930 - val_accuracy: 0.9242 - val_loss: 0.1994
Epoch 4/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 316s 4s/step - accuracy: 0.9808 - loss: 0.0617 - val_accuracy: 0.9470 - val_loss: 0.1515
Epoch 5/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 319s 4s/step - accuracy: 0.9781 - loss: 0.0711 - val_accuracy: 0.9394 - val_loss: 0.1652
Epoch 6/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 308s 4s/step - accuracy: 0.9848 - loss: 0.0451 - val_accuracy: 0.9470 - val_loss: 0.1317
Epoch 7/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 312s 4s/step - accuracy: 0.9873 - loss: 0.0427 - val_accuracy: 0.9754 - val_loss: 0.0804
Epoch 8/30
75/75 ━━━━━━━━━━━━━━━━━━━━ 313s 4s/step - accuracy: 0.9889 - loss: 0.0363 - val_accuracy: 0.9773 - val_loss: 0.

model.save("xception_model.keras")

In [10]:
model.save("xception_model.keras")

In [11]:
model.save("xception_model_h5.h5")

In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 150, 150, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ xception (Functional)                │ (None, 2048)                │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │           4,098 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 62,487,680 (238.37 MB)

 Trainable params: 20,811,050 (79.39 MB)

 Non-trainable params: 54,528 (213.00 KB)

 Optimizer params: 41,622,102 (158.78 MB)

In [13]:
import joblib

In [14]:
joblib.dump(model, "xception_model_pkl.pkl")

['xception_model_pkl.pkl']